## Data Description

You are provided with 25,010 poker hands in train.csv and 1,000,000 in test.csv. Each hand consists of five cards with a given suit and rank, drawn from a standard deck of 52. Suits and ranks are represented as ordinal categories:

```
S1 “Suit of card #1”
Ordinal (1-4) representing {Hearts, Spades, Diamonds, Clubs}
C1 “Rank of card #1”
Numerical (1-13) representing (Ace, 2, 3, ... , Queen, King)

...

S5 “Suit of card #5”
C5 “Rank of card #5”
```

Each row in the training set has the accompanying class label for the poker hand it comprises. The hands are omitted from the test set and must be predicted by participants. Hands are classified into the following ordinal categories:


```
0: Nothing in hand; not a recognized poker hand 
1: One pair; one pair of equal ranks within five cards
2: Two pairs; two pairs of equal ranks within five cards
3: Three of a kind; three equal ranks within five cards
4: Straight; five cards, sequentially ranked with no gaps
5: Flush; five cards with the same suit
6: Full house; pair + different rank three of a kind
7: Four of a kind; four equal ranks within five cards
8: Straight flush; straight + flush
9: Royal flush; {Ace, King, Queen, Jack, Ten} + flush
```
Note that the Straight flush and Royal flush hands are not representative of
the true domain because they have been over-sampled. The straight flush
is 14.43 times more likely to occur in the training set, while the royal flush is 129.82 times more likely.

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from functools import reduce
import random

## Utils

#### Test function

In [2]:
def calc_hand_label(hand):    
    def f(hand):
        suits_hist = reduce(lambda d,x: {**d, **{x: (d.get(x, 0) + 1)}}, hand[:,0], {})
        ranks_hist = reduce(lambda d,x: {**d, **{x: (d.get(x, 0) + 1)}}, hand[:,1], {})
        
        if len(ranks_hist.values()) < 5:
            if len(ranks_hist.values()) == 2:
                if max(list(ranks_hist.values())) == 4:
                    return 7
    #                 print('7: Four of a kind; four equal ranks within five cards')
                else:
                    return 6
    #                 print('6 Full house; pair + different rank three of a kind')
            elif len(ranks_hist.values()) == 3:
                if max(list(ranks_hist.values())) == 3:
                    return 3
    #                 print('3: Three of a kind; three equal ranks within five cards')
                else:
                    return 2
    #                 print('2: Two pairs; two pairs of equal ranks within five cards')
            else:
                return 1
    #             print('1: One pair; one pair of equal ranks within five cards')
        else:
            if len(suits_hist.values()) == 1:
                if max(list(ranks_hist.keys())) -  min(list(ranks_hist.keys())) == 4:
                    if max(list(ranks_hist.keys())) == 13:
                        return 9
    #                     print('9: Royal flush; {Ace, King, Queen, Jack, Ten} + flush')
                    else:
                        return 8
    #                     print('8: Straight flush; straight + flush')
                else:
                    return 5
    #                 print('5: Flush; five cards with the same suit')
            elif max(list(ranks_hist.keys())) -  min(list(ranks_hist.keys())) == 4:
                return 4    
    #             print('4: Straight; five cards, sequentially ranked with no gaps')
            else:
                return 0
                print('0: Nothing in hand; not a recognized poker hand ')
                
    hand = np.array(hand).reshape(5,2)
    res1 = f(hand)
    
    hand[:,1] = list(map(lambda x: 13 if (x == 1) else x-1, hand[:,1]))
    res2 = f(hand)
    
    if res1 == 9 and res2 == 8:
        return res2
    else:
        return max(res1, res2)

#### Encode cards

In [3]:
def card_encode(s, c):
    cards=["A","2","3","4","5","6","7","8","9","10","J","Q","K"]
    suits="♥♦♣♠"
    return cards[c - 1] + suits[s - 1]

def hand_encode(hand):
    return ', '.join(map(lambda x: card_encode(hand[x*2], hand[x*2+1]),range(5)))

print(hand_encode([4,9,2,1,2,2,4,7,2,8]))
print(hand_encode([2,9,2,4,3,6,1,9,4,9]))

9♠, A♦, 2♦, 7♠, 8♦
9♦, 4♦, 6♣, 9♥, 9♠


## Train data

In [4]:
def shuffle_df(data):
    data = data.copy()
    for index, row in data.iterrows():
        rank = -1
    
        row['S1'] = random.randint(1,4)
        row['S2'] = random.randint(1,4)
        row['S3'] = random.randint(1,4)
        row['S4'] = random.randint(1,4)
        row['S5'] = random.randint(1,4)
        row['C1'] = random.randint(1,13)
        row['C2'] = random.randint(1,13)
        row['C3'] = random.randint(1,13)
        row['C4'] = random.randint(1,13)
        row['C5'] = random.randint(1,13)
        row['hand'] = calc_hand_label(row[['S1', 'C1','S2', 'C2','S3', 'C3','S4', 'C4','S5', 'C5']])
    return data

In [5]:
suits = ['S1','S2','S3','S4','S5']
cards = ['C1','C2','C3','C4','C5']
distances = ['D1', 'D2', 'D3', 'D4', 'D5']

In [6]:
train_dataset = pd.read_csv('train.csv', delimiter=',')

In [7]:
train_dataset = train_dataset\
.append(shuffle_df(train_dataset),ignore_index=True)\
.append(shuffle_df(train_dataset),ignore_index=True)\
.append(shuffle_df(train_dataset),ignore_index=True)\

In [28]:
train_dataset.to_csv('./big_train_dataset.csv')

In [29]:
train_dataset = pd.read_csv('./big_train_dataset.csv').drop('Unnamed: 0', axis=1)

In [30]:
train_dataset.shape

(100040, 11)

In [9]:
data = train_dataset.drop('hand', axis=1)
train_labels = train_dataset['hand']

In [10]:
labels_vect = tf.keras.utils.to_categorical(train_labels)

#### Adding distance features

In [5]:
def get_distances(hand):
    distances_list = []
    for i, card in enumerate(hand):
        if (i < len(hand) - 1):
            distances_list.append(hand[i+1] - card)
        else:
            distances_list.append(13 - card + hand[0])
    return pd.Series(distances_list)

def with_distances(data):
    sorted = data[cards]
    sorted.values.sort()

    distances_df = sorted.apply(get_distances, axis=1)
    distances_df.columns = distances
    
    data_with_distances = pd.concat([data, distances_df], axis=1)
    return data_with_distances

In [12]:
data_with_distances = with_distances(data)
data_with_distances.head()

,S1,C1,S2,C2,S3,C3,S4,C4,S5,C5,D1,D2,D3,D4,D5
0,4,9,2,1,2,2,4,7,2,8,1,5,1,1,5
1,1,4,3,6,1,12,3,11,2,7,2,1,4,1,5
2,1,11,4,1,3,7,4,11,2,1,0,6,4,0,3
3,2,9,2,4,3,6,1,9,4,9,2,3,0,0,8
4,1,8,2,4,2,11,2,2,2,1,1,2,4,3,3


#### One hot encoding

In [6]:
def one_hot_encoded(data, columns, encoder, mode='transform'):
    encoded = None
    if (mode == 'fit'):
        encoded = encoder.fit_transform(data[columns])
    elif (mode == 'transform'):
        encoded = encoder.transform(data[columns])
    return pd.concat([data.drop(columns, axis=1), pd.DataFrame(encoded.toarray())], axis=1)

In [14]:
encoder = OneHotEncoder()
data_encoded = one_hot_encoded(data_with_distances, suits, encoder, 'fit')
print(data_encoded.shape)

data_encoded.head()

(100040, 30)


,C1,C2,C3,C4,C5,D1,D2,D3,D4,D5,...,10,11,12,13,14,15,16,17,18,19
0,9,1,2,7,8,1,5,1,1,5,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,4,6,12,11,7,2,1,4,1,5,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,11,1,7,11,1,0,6,4,0,3,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,9,4,6,9,9,2,3,0,0,8,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,8,4,11,2,1,1,2,4,3,3,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


#### Normalization

In [7]:
def normalized(data, columns, mean, std):
    data_normalized = (data[columns] - mean) / std
    to_return = pd.concat([data_normalized, data.drop(columns, axis=1)], axis=1)
    return to_return

In [16]:
mean_c = data_encoded[cards].mean().mean()
std_c = pd.Series(data_encoded[cards].values.flatten()).std()
mean_d = data_encoded[distances].mean().mean()
std_d = pd.Series(data_encoded[distances].values.flatten()).std()

data_normalized = normalized(data_encoded, cards, mean_c, std_c)
data_normalized = normalized(data_normalized, distances, mean_d, std_d)
data_normalized.head()

,D1,D2,D3,D4,D5,C1,C2,C3,C4,C5,...,10,11,12,13,14,15,16,17,18,19
0,-0.748732,1.123098,-0.748732,-0.748732,1.123098,0.534380,-1.603162,-1.335969,-0.000006,0.267187,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,-0.280775,-0.748732,0.655141,-0.748732,1.123098,-0.801584,-0.267199,1.335958,1.068765,-0.000006,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,-1.216690,1.591056,0.655141,-1.216690,0.187183,1.068765,-1.603162,-0.000006,1.068765,-1.603162,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,-0.280775,0.187183,-1.216690,-1.216690,2.526971,0.534380,-0.801584,-0.267199,0.534380,0.534380,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,-0.748732,-0.280775,0.655141,0.187183,0.187183,0.267187,-0.801584,1.068765,-1.335969,-1.603162,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


In [17]:
data_normalized.shape

(100040, 30)

## Neural Network

In [18]:
from time import time
from keras.callbacks import TensorBoard

model = keras.Sequential()
#input
model.add(keras.layers.Dense(120, activation='relu', input_shape=(30,)))
model.add(keras.layers.Dropout(0.2))
#hidden
model.add(keras.layers.Dense(240, activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(120, activation='relu'))
model.add(keras.layers.Dropout(0.1))
model.add(keras.layers.Dense(120, activation='relu'))
model.add(keras.layers.Dense(60, activation='relu'))
# Add a softmax layer with 10 output units:
model.add(keras.layers.Dense(10, activation='softmax'))

model.compile(optimizer=tf.train.AdamOptimizer(0.0005),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

tensorboard = TensorBoard(log_dir="logs/{}".format(time()))

model.fit(np.array(data_normalized), np.array(labels_vect), epochs=300, batch_size=512)

Using TensorFlow backend.


Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Epoch 1/300
100040/100040 [==============================]100040/100040 [==============================] - 3s 25us/step - loss: 1.1021 - acc: 0.5346

Epoch 2/300
100040/100040 [==============================]100040/100040 [==============================] - 2s 24us/step - loss: 0.7023 - acc: 0.7166

Epoch 3/300
100040/100040 [==============================]100040/100040 [==============================] - 2s 25us/step - loss: 0.2959 - acc: 0.9000

Epoch 4/300
100040/100040 [==============================]100040/100040 [==============================] - 3s 26us/step - loss: 0.1031 - acc: 0.9713

Epoch 5/300
100040/100040 [==============================]100040/100040 [==============================] - 2s 25us/step - loss: 0.0594 - acc: 0.9855

Epoch 6/300
100040/100040 [==============================]100040/100040 [==============================]

100040/100040 [==============================]100040/100040 [==============================] - 2s 24us/step - loss: 8.8917e-04 - acc: 0.9998

Epoch 103/300
100040/100040 [==============================]100040/100040 [==============================] - 2s 24us/step - loss: 3.5965e-04 - acc: 0.9999

Epoch 104/300
100040/100040 [==============================]100040/100040 [==============================] - 3s 25us/step - loss: 7.6502e-04 - acc: 0.9998

Epoch 105/300
100040/100040 [==============================]100040/100040 [==============================] - 2s 24us/step - loss: 5.4247e-04 - acc: 0.9998

Epoch 106/300
100040/100040 [==============================]100040/100040 [==============================] - 3s 26us/step - loss: 6.6645e-04 - acc: 0.9998

Epoch 107/300
100040/100040 [==============================]100040/100040 [==============================] - 2s 24us/step - loss: 6.2446e-04 - acc: 0.9998

Epoch 108/300
100040/100040 [==============================]100040/100040 [===

100040/100040 [==============================]100040/100040 [==============================] - 3s 32us/step - loss: 5.6129e-04 - acc: 0.9999

Epoch 207/300
100040/100040 [==============================]100040/100040 [==============================] - 3s 33us/step - loss: 3.2588e-04 - acc: 0.9999

Epoch 208/300
100040/100040 [==============================]100040/100040 [==============================] - 3s 28us/step - loss: 3.9712e-04 - acc: 0.9999

Epoch 209/300
100040/100040 [==============================]100040/100040 [==============================] - 3s 28us/step - loss: 1.8343e-04 - acc: 1.0000

Epoch 210/300
100040/100040 [==============================]100040/100040 [==============================] - 3s 29us/step - loss: 5.1505e-05 - acc: 1.0000

Epoch 211/300
100040/100040 [==============================]100040/100040 [==============================] - 3s 33us/step - loss: 3.7561e-04 - acc: 0.9999

Epoch 212/300
100040/100040 [==============================]100040/100040 [===

In [27]:
model.save('./model-5')

## Test data

In [13]:
test_dataset = pd.read_csv('test.csv', delimiter=',')
test_data = test_dataset.drop('id', axis=1)

In [15]:
test_labels = test_data.apply(calc_hand_label, axis=1)

In [20]:
test_data_with_distances = with_distances(test_data)
test_data_encoded = one_hot_encoded(test_data_with_distances, suits, encoder)
test_data_normalized = normalized(test_data_encoded, cards, mean_c, std_c)
test_data_normalized = normalized(test_data_normalized, distances, mean_d, std_d)

In [21]:
test_data_normalized.shape

(1000000, 30)

In [30]:
test_data_normalized.to_csv('./test_data_with_distances.csv')

In [8]:
test_data_normalized = pd.read_csv('./test_data_with_distances.csv').drop('Unnamed: 0', axis=1)

In [9]:
test_data_normalized.head()

,D1,D2,D3,D4,D5,C1,C2,C3,C4,C5,...,10,11,12,13,14,15,16,17,18,19
0,-0.744891,-0.744891,1.117337,-0.279334,0.651780,0.803039,-1.331849,-1.064988,0.269317,-1.598710,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
1,-0.744891,-0.744891,-0.744891,1.582894,0.651780,1.603622,-0.531266,0.002456,-0.264405,-0.798127,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
2,-0.279334,-0.744891,0.651780,0.186223,0.186223,-1.064988,1.069900,0.269317,-1.598710,-0.798127,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.186223,-0.744891,-0.744891,0.186223,1.117337,-0.264405,-1.064988,0.002456,0.269317,1.069900,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,-0.279334,-1.210448,0.651780,-0.279334,1.117337,0.803039,-0.798127,-0.264405,1.336761,-0.264405,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


## Predictions

In [10]:
model = keras.models.load_model('./model-5')
model.compile(optimizer=tf.train.AdamOptimizer(0.0005),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [11]:
test_data_sample_predicted_10 = model.predict(np.array(test_data_normalized))
test_data_sample_predicted = np.array(list(map(np.argmax, test_data_sample_predicted_10)))

In [16]:
predictions_data = pd.concat([
    test_dataset,
    pd.DataFrame({'predictions': test_data_sample_predicted}),
    pd.DataFrame({'labels': test_labels}),
], axis=1)

In [17]:
wrong_answers = predictions_data[predictions_data['predictions'] != predictions_data['labels']]
wrong_answers.shape

(5, 13)

In [18]:
wrong_answers['labels'].value_counts()

5    2
0    2
4    1
Name: labels, dtype: int64

In [19]:
wrong_answers

,id,S1,C1,S2,C2,S3,C3,S4,C4,S5,C5,predictions,labels
79985,79986,4,13,2,10,4,12,4,11,3,1,1,4
173705,173706,4,10,2,13,4,12,4,3,3,11,1,0
677907,677908,3,1,3,11,3,10,3,9,3,12,9,5
816243,816244,1,11,1,2,1,12,1,10,1,13,9,5
866415,866416,4,11,2,12,4,13,4,2,3,10,1,0


In [20]:
wrong_answers.drop(['id', 'predictions', 'labels'], axis=1).apply(lambda h: hand_encode(np.array(h)), axis=1)

79985     K♠, 10♦, Q♠, J♠, A♣
173705    10♠, K♦, Q♠, 3♠, J♣
677907    A♣, J♣, 10♣, 9♣, Q♣
816243    J♥, 2♥, Q♥, 10♥, K♥
866415    J♠, Q♦, K♠, 2♠, 10♣
dtype: object

## Submission

In [21]:
submission = predictions_data[['id', 'predictions']]
submission.to_csv('./kaggle_submission.csv')